# Q1. Embedding the query

Embed the query: `'I just discovered the course. Can I join now?'`. Use the `'jinaai/jina-embeddings-v2-small-en'` model.

You should get a numpy array of size 512.

What's the minimal value in this array?
- -0.51
- -0.11
- 0
- 0.51

In [1]:
from fastembed import TextEmbedding

documents = ['I just discovered the course. Can I join now?']
model = TextEmbedding(model_name='jinaai/jina-embeddings-v2-small-en')
embeddings_list = list(model.embed(documents))
min(embeddings_list[0])

np.float64(-0.11726372517101391)

**Answer:** `-0.11`

## Cosine similarity

The vectors that our embedding model returns are already normalized: their length is 1.0.

You can check that by using the `norm` function:

```python
import numpy as np
np.linalg.norm(q)
```

Which means that we can simply compute the dot product between two vectors to learn the cosine similarity between them.

For example, if you compute the cosine of the query vector with itself, the result will be 1.0:
```python
q.dot(q)
```

___


# Q2. Cosine similarity with another vector

Now let's embed this document:
```
doc = 'Can I still join the course after the start date?'
```

What's the cosine similarity between the vector for the query and the vector for the document?
- 0.3
- 0.5
- 0.7
- 0.9

In [2]:
documents = [
    'I just discovered the course. Can I join now?',
    'Can I still join the course after the start date?'
]
vectors = list(model.embed(documents))
vectors[0].dot(vectors[1])

np.float64(0.9008529111486461)

**Answer:** `0.9`

---

# Q3. Ranking by cosine

For Q3 and Q4 we will use these documents:
```python
documents = [{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
  'section': 'General course-related questions',
  'question': 'Course - Can I follow the course after it finishes?',
  'course': 'data-engineering-zoomcamp'},
 {'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
  'section': 'General course-related questions',
  'question': 'Course - When will the course start?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'You can start by installing and setting up all the dependencies and requirements:\nGoogle cloud account\nGoogle Cloud SDK\nPython 3 (installed with Anaconda)\nTerraform\nGit\nLook over the prerequisites and syllabus to see if you are comfortable with these subjects.',
  'section': 'General course-related questions',
  'question': 'Course - What can I do before the course starts?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Star the repo! Share it with friends if you find it useful ❣️\nCreate a PR if you see you can improve the text or the structure of the repository.',
  'section': 'General course-related questions',
  'question': 'How can we contribute to the course?',
  'course': 'data-engineering-zoomcamp'}]
```

Compute the embeddings for the text field, and compute the cosine between the query vector and all the documents.

What's the document index with the highest similarity? (Indexing starts from 0):

- 0
- 1
- 2
- 3
- 4

In [9]:
base_doc = ['I just discovered the course. Can I join now?']
base_vector = list(model.embed(base_doc))[0]

other_docs = [{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
  'section': 'General course-related questions',
  'question': 'Course - Can I follow the course after it finishes?',
  'course': 'data-engineering-zoomcamp'},
 {'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
  'section': 'General course-related questions',
  'question': 'Course - When will the course start?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'You can start by installing and setting up all the dependencies and requirements:\nGoogle cloud account\nGoogle Cloud SDK\nPython 3 (installed with Anaconda)\nTerraform\nGit\nLook over the prerequisites and syllabus to see if you are comfortable with these subjects.',
  'section': 'General course-related questions',
  'question': 'Course - What can I do before the course starts?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Star the repo! Share it with friends if you find it useful ❣️\nCreate a PR if you see you can improve the text or the structure of the repository.',
  'section': 'General course-related questions',
  'question': 'How can we contribute to the course?',
  'course': 'data-engineering-zoomcamp'}]

other_docs = [doc['text'] for doc in other_docs]
other_vectors = list(model.embed(other_docs))
[base_vector.dot(other_vector) for other_vector in other_vectors]

[np.float64(0.7629684779447995),
 np.float64(0.8182378328589502),
 np.float64(0.8085397403172203),
 np.float64(0.7133078762855278),
 np.float64(0.7304499352679792)]

**Answer:** `1`

---

# Q4. Ranking by cosine, version two

Now let's calculate a new field, which is a concatenation of `question` and `text`:
```
full_text = doc['question'] + ' ' + doc['text']
```

Embed this field and compute the cosine between it and the query vector. What's the highest scoring document?
- 0
- 1
- 2
- 3
- 4

Is it different from Q3? If yes, why?

In [12]:
base_doc = ['I just discovered the course. Can I join now?']
base_vector = list(model.embed(base_doc))[0]

other_docs = [{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
  'section': 'General course-related questions',
  'question': 'Course - Can I follow the course after it finishes?',
  'course': 'data-engineering-zoomcamp'},
 {'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
  'section': 'General course-related questions',
  'question': 'Course - When will the course start?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'You can start by installing and setting up all the dependencies and requirements:\nGoogle cloud account\nGoogle Cloud SDK\nPython 3 (installed with Anaconda)\nTerraform\nGit\nLook over the prerequisites and syllabus to see if you are comfortable with these subjects.',
  'section': 'General course-related questions',
  'question': 'Course - What can I do before the course starts?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Star the repo! Share it with friends if you find it useful ❣️\nCreate a PR if you see you can improve the text or the structure of the repository.',
  'section': 'General course-related questions',
  'question': 'How can we contribute to the course?',
  'course': 'data-engineering-zoomcamp'}]

other_docs = [doc['question'] + ' ' + doc['text'] for doc in other_docs]
other_vectors = list(model.embed(other_docs))
[base_vector.dot(other_vector) for other_vector in other_vectors]

[np.float64(0.8514543367988866),
 np.float64(0.8436594295942044),
 np.float64(0.8408287244127188),
 np.float64(0.7755157723274342),
 np.float64(0.8086008208120041)]

**Answer:** `0`
It is different from Q3 because the data has been enriched by its question. It increases the accuracy.

---

# Q5. Selecting the embedding model

Now let's select a smaller embedding model. What's the smallest dimensionality for models in fastembed?
- 128
- 256
- 384
- 512

One of these models is `BAAI/bge-small-en`. Let's use it.

In [17]:
from fastembed import TextEmbedding
min([model['dim'] for model in TextEmbedding.list_supported_models()])

384

**Answer:**: `384`

---

# Q6. Indexing with qdrant

For the last question, we will use more documents.

We will select only FAQ records from our ml zoomcamp:

```python
import requests 

docs_url = 'https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()


documents = []

for course in documents_raw:
    course_name = course['course']
    if course_name != 'machine-learning-zoomcamp':
        continue

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)
```

Add them to qdrant using the model from Q5.

When adding the data, use both question and answer fields:

```python
text = doc['question'] + ' ' + doc['text']
```

After the data is inserted, use the question from Q1 for querying the collection.

What's the highest score in the results? (The score for the first returned record):

- 0.97
- 0.87
- 0.77
- 0.67



In [26]:
import requests 

from qdrant_client import QdrantClient, models

EMBEDDING_DIMENSIONALITY = 384
collection_name = 'zoomcamp-rag'

client = QdrantClient("http://qdrant:6333")
if not client.get_collection(collection_name):
    client.create_collection(
        collection_name=collection_name,
        vectors_config=models.VectorParams(
            size=EMBEDDING_DIMENSIONALITY,
            distance=models.Distance.COSINE
        )
    )

docs_url = 'https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

id = 0
points = []
model_handle = 'BAAI/bge-small-en'

for course in documents_raw:
    course_name = course['course']
    if course_name != 'machine-learning-zoomcamp':
        continue

    for doc in course['documents']:
        text = doc['question'] + ' ' + doc['text']
        point = models.PointStruct(
            id=id,
            vector=models.Document(
                text=text,
                model=model_handle
            ),
            payload={
                'text': text,
                'section': doc['section'],
                'course': course_name
            }
        )
        points.append(point)

        id +=1

client.upsert(
    collection_name=collection_name,
    points=points
)


Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

tokenizer.json: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/701 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

model_optimized.onnx:   0%|          | 0.00/133M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

UpdateResult(operation_id=0, status=<UpdateStatus.COMPLETED: 'completed'>)

In [34]:
results = client.query_points(
    collection_name=collection_name,
    query=models.Document(
        text='I just discovered the course. Can I join now?',
        model=model_handle
    ),
    limit=1,
    with_payload=True
)
print(results[0])

TypeError: 'QueryResponse' object is not subscriptable